# 汇总conv1d/conv2d/LSTM/RNN/GRU/BiLSTM模型，多次迭代取预测结果的平均值

In [1]:
# 导入包
# This makes it so that matplotlib graphics will show up within the Jupyter Notebook.
%matplotlib inline

# Standard library import

# Data Analysis Tools
import pandas as pd
import numpy as np

# Visualization Tools
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Tools
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# File Loading Tools 
import os
import pickle

# Deep Learning Tools
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Flatten, Dropout
from keras.layers import Convolution1D, MaxPooling1D
from keras.layers import Conv2D, MaxPool2D
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [2]:
# Load Data 
X_train = np.load("./data phm 2016/X_train_r_modeI_chamber4_mm.npy")
y_train = np.load("./data phm 2016/y_train_modeI_chamber4_mm.npy")
X_test = np.load("./data phm 2016/X_test_r_modeI_chamber4_mm.npy")
y_test = np.load("./data phm 2016/y_test_modeI_chamber4_mm.npy")
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

# Basic parameter
wafer_number, max_batch_length, variable_number = X_train.shape
wafer_number_test = X_test.shape[0]
print('训练集晶圆个数：', wafer_number)
print('最长时间序列长度：', max_batch_length)
print('字段个数：', variable_number)
print('训练集晶圆个数：', wafer_number_test)

X_train shape:  (798, 263, 19)
y_train shape:  (798, 1)
X_test shape:  (165, 263, 19)
y_test shape:  (165, 1)
训练集晶圆个数： 798
最长时间序列长度： 263
字段个数： 19
训练集晶圆个数： 165


In [3]:
# reshape X for conv2d
X_train_r = X_train.reshape((wafer_number, max_batch_length, variable_number, 1))
X_test_r = X_test.reshape((wafer_number_test, max_batch_length, variable_number, 1))
print('X_train_r shape: ', X_train_r.shape)
print('X_test_r shape: ', X_test_r.shape)

X_train_r shape:  (798, 263, 19, 1)
X_test_r shape:  (165, 263, 19, 1)


In [4]:
ss = StandardScaler()
y_train_ss = ss.fit_transform(y_train)

In [5]:
def predict_conv1d():
    # Step1:define net
    model = Sequential(name='model_conv1d')
    model.add(Convolution1D(filters=256, kernel_size=2, activation='relu', input_shape=(max_batch_length, variable_number), name='conv1d1'))
    model.add(MaxPooling1D(8, name='maxpool1'))
    model.add(Convolution1D(filters=128, kernel_size=2, activation='relu', name='conv1d2'))
    model.add(MaxPooling1D(8, name='maxpool2'))
    model.add(Convolution1D(filters=64, kernel_size=2, activation='relu', name='conv1d3'))
    model.add(MaxPooling1D(2, name='maxpool3'))

    model.add(Flatten(name='flatten'))

    model.add(Dense(16, activation='linear', name='dense1'))
    model.add(Dense(1, activation='linear', name='dense2'))
    print('Conv1d model define done')
        
    # Step2:compile
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    
    # Step3:fit
    history_model_conv1d = model.fit(X_train, y_train, epochs=400, batch_size=200) #epoch400
    print('Conv1d model fit done')
    
    # Step4:predict test set
    y_test_pre = model.predict(X_test)
    print('Predict test set done')
    
    # Step5:compute metrics
    mse = mean_squared_error(y_test, y_test_pre)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pre))
    mae = mean_absolute_error(y_test, y_test_pre)
    r2 = r2_score(y_test, y_test_pre)
    print('Compute metrics done')
    
    return y_test_pre, mse, rmse, mae, r2, model

In [6]:
def predict_conv2d():
    # Step1:define model
    model = Sequential(name='model_conv2d')

    model.add(Conv2D(filters=16, kernel_size=[2, 2], padding='same', input_shape=(max_batch_length, variable_number, 1)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Conv2D(filters=32, kernel_size=[2, 2], padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Conv2D(filters=64, kernel_size=[2, 2], padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Conv2D(filters=16, kernel_size=[2, 2], padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))

    model.add(Dense(16))
    model.add(Activation('relu'))

    model.add(Dense(1))
    model.add(Activation('linear'))
    
    print('Conv2d model define done')
    
    # Step2:compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    nb_epoch = 1000
    batch_size = 399

    Reduce=ReduceLROnPlateau(monitor='mse',
                             factor=0.5,
                             patience=10,
                             verbose=1,
                             mode='min',
                             min_delta=0.0001,
                             cooldown=0,
                             min_lr=1e-10)
    
    # Step3:fit
    model.fit(x=X_train_r, y=y_train, epochs=nb_epoch, batch_size=batch_size, callbacks=[Reduce])
    print('Conv1d model fit done')
    
    # Step4:predict test set
    y_test_pre = model.predict(X_test_r)
    print('Predict test set done')
    
    # Step5:compute metrics
    mse = mean_squared_error(y_test, y_test_pre)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pre))
    mae = mean_absolute_error(y_test, y_test_pre)
    r2 = r2_score(y_test, y_test_pre)
    print('Compute metrics done')
    
    return y_test_pre, mse, rmse, mae, r2, model

In [7]:
def predict_lstm():
    # Step1:define model
    model_lstm = Sequential(name='model_lstm')
    model_lstm.add(LSTM(input_shape=(max_batch_length,variable_number), units=40, activation='tanh', return_sequences=True, name='lstm1'))
    model_lstm.add(LSTM(units=40, activation='tanh', return_sequences=False, name='lstm2'))

    model_lstm.add(Dense(4, name='dense1'))
    model_lstm.add(Dense(1, name='dense2'))
    
    print('LSTM model define done')
    
    # Step2:compile model
    adam = Adam(learning_rate=0.001)
    model_lstm.compile(loss='mse', optimizer=adam, metrics=['mse'])

    nb_epoch = 400
    batch_size = 200
    
    # Step3:fit
    history_lstm = model_lstm.fit(X_train, y_train_ss, epochs=nb_epoch, batch_size=batch_size) 
    print('LSTM model fit done')
    
    # Step4:predict test set
    y_test_pre = model_lstm.predict(X_test)
    y_test_pre = ss.inverse_transform(y_test_pre)
    print('Predict test set done')
    
    # Step5:compute metrics
    mse = mean_squared_error(y_test, y_test_pre)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pre))
    mae = mean_absolute_error(y_test, y_test_pre)
    r2 = r2_score(y_test, y_test_pre)
    print('Compute metrics done')
    
    return y_test_pre, mse, rmse, mae, r2, model_lstm

In [8]:
def predict_rnn():
    # Step1:define model
    model_rnn = Sequential(name='model_rnn')
    model_rnn.add(SimpleRNN(input_shape=(max_batch_length,variable_number), units=40, activation='tanh', return_sequences=True))
    model_rnn.add(SimpleRNN(units=40, activation='tanh', return_sequences=False))

    model_rnn.add(Dense(4))
    model_rnn.add(Dense(1))
    print('RNN model define done')
    
    # Step2:compile model
    adam = Adam(learning_rate=0.001)
    model_rnn.compile(loss='mse', optimizer=adam, metrics=['mse']) # rmsprop

    nb_epoch = 100
    batch_size = 200
    
    # Step3:fit
    model_rnn.fit(X_train, y_train_ss, epochs=nb_epoch, batch_size=batch_size)
    print('RNN model fit done')
    
    # Step4:predict test set
    y_test_pre = model_rnn.predict(X_test)
    y_test_pre = ss.inverse_transform(y_test_pre)
    print('Predict test set done')
    
    # Step5:compute metrics
    mse = mean_squared_error(y_test, y_test_pre)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pre))
    mae = mean_absolute_error(y_test, y_test_pre)
    r2 = r2_score(y_test, y_test_pre)
    print('Compute metrics done')
    
    return y_test_pre, mse, rmse, mae, r2, model_rnn

In [9]:
def predict_gru():
    # Step1:define model
    model_gru = Sequential(name='model_GRU')
    model_gru.add(GRU(input_shape=(max_batch_length,variable_number), units=40, activation='relu', return_sequences=True))
    model_gru.add(GRU(units=40, activation='relu', return_sequences=False))

    model_gru.add(Dense(4))
    model_gru.add(Dense(1))
    
    print('GRU model define done')
    
    # Step2:compile model
    adam = Adam(learning_rate=0.001)
    model_gru.compile(loss='mse', optimizer=adam, metrics=['mse']) # rmsprop

    nb_epoch = 150
    batch_size = 100

    # Step3:fit
    model_gru.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size)
    print('GRU model fit done')
    
    # Step4:predict test set
    y_test_pre = model_gru.predict(X_test)
    print('Predict test set done')
    
    # Step5:compute metrics
    mse = mean_squared_error(y_test, y_test_pre)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pre))
    mae = mean_absolute_error(y_test, y_test_pre)
    r2 = r2_score(y_test, y_test_pre)
    print('Compute metrics done')
    
    return y_test_pre, mse, rmse, mae, r2, model_gru

In [10]:
def predict_bilstm():
    # Step1:define model
    model_biLSTM = Sequential(name='model_biLSTM')
    model_biLSTM.add(Bidirectional(LSTM(units=40, activation='tanh', return_sequences=True), input_shape=(max_batch_length,variable_number), name='biLSTM1'))
    model_biLSTM.add(Bidirectional(LSTM(units=40, activation='tanh', return_sequences=False), name='biLSTM2'))

    model_biLSTM.add(Dense(4, name='dense1'))
    model_biLSTM.add(Dense(1, name='dense2'))

    print('BiLSTM model define done')
    
    # Step2:compile model
    adam = Adam(learning_rate=0.001)
    model_biLSTM.compile(loss='mse', optimizer=adam, metrics=['mse'])
    nb_epoch = 50
    batch_size = 200

    # Step3:fit
    history_biLSTM = model_biLSTM.fit(X_train, y_train_ss, epochs=nb_epoch, batch_size=batch_size) 
    print('BiLSTM model fit done')
    
    # Step4:predict test set
    y_test_pre = model_biLSTM.predict(X_test)
    y_test_pre = ss.inverse_transform(y_test_pre)
    print('Predict test set done')
    
    # Step5:compute metrics
    mse = mean_squared_error(y_test, y_test_pre)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pre))
    mae = mean_absolute_error(y_test, y_test_pre)
    r2 = r2_score(y_test, y_test_pre)
    print('Compute metrics done')
    
    return y_test_pre, mse, rmse, mae, r2, model_biLSTM

In [11]:
def deep_learning_iteration(model_name, N_iter=10):
    y_test_pre_list = []
    mse_list, rmse_list, mae_list, r2_list = [], [], [], []
    print('*******************')
    for i in range(0, N_iter):
        print(f'Iter {i} Start:')
        y_test_pre, mse, rmse, mae, r2, model = eval(f'predict_{model_name}')()

        y_test_pre_list.append(y_test_pre)
        mse_list.append(mse); rmse_list.append(rmse); mae_list.append(mae); r2_list.append(r2)

    y_test_pre_mean = np.mean(y_test_pre_list, axis=0)
    mse_mean, rmse_mean, mae_mean, r2_mean = np.mean(mse_list), np.mean(rmse_list), np.mean(mae_list), np.mean(r2_list)

    print(f'Mean results of {N_iter} times iteration:')
    print(f'Mean mse of {N_iter} times iteration:', mse_mean)
    print(f'Mean rmse of {N_iter} times iteration:', rmse_mean)
    print(f'Mean mae of {N_iter} times iteration:', mae)
    print(f'Mean r2 of {N_iter} times iteration:', r2_mean)

    dict_result = {}
    dict_result = {'y_test_pre':y_test_pre_mean, 'mse':mse_mean, 'rmse':r2_mean, 'mae':mae_mean, 'r2':r2_mean}

    pickle.dump(dict_result, open(f'./results_save/results_{model_name}.pkl', 'wb'))
    # save last model
    model.save(f'./results_save/model_{model_name}.h5')

In [12]:
deep_learning_iteration(model_name='conv1d', N_iter=2)

*******************
Iter 0 Start:
Conv1d model define done
Epoch 1/2
4/4 [==============================] - 1s 93ms/step - loss: 5248.1206 - mse: 5248.1206
Epoch 2/2
4/4 [==============================] - 0s 104ms/step - loss: 4797.8042 - mse: 4797.8047
Conv1d model fit done
Predict test set done
Compute metrics done
Iter 1 Start:
Conv1d model define done
Epoch 1/2
4/4 [==============================] - 1s 96ms/step - loss: 5261.8193 - mse: 5261.8193
Epoch 2/2
4/4 [==============================] - 0s 96ms/step - loss: 4818.4497 - mse: 4818.4492
Conv1d model fit done
Predict test set done
Compute metrics done
Mean results of 2 times iteration:
Mean mse of 2 times iteration: 4247.7960786893545
Mean rmse of 2 times iteration: 65.17508567776616
Mean mae of 2 times iteration: 64.9310098841858
Mean r2 of 2 times iteration: -102.04265622910519


In [13]:
deep_learning_iteration(model_name='conv2d', N_iter=2)

*******************
Iter 0 Start:
Conv2d model define done
Epoch 1/2
2/2 [==============================] - 1s 415ms/step - loss: 5378.7656 - mse: 5378.7656 - lr: 0.0010
Epoch 2/2
2/2 [==============================] - 1s 389ms/step - loss: 5356.7056 - mse: 5356.7056 - lr: 0.0010
Conv1d model fit done
Predict test set done
Compute metrics done
Iter 1 Start:
Conv2d model define done
Epoch 1/2
2/2 [==============================] - 1s 394ms/step - loss: 5377.0967 - mse: 5377.0967 - lr: 0.0010
Epoch 2/2
2/2 [==============================] - 1s 409ms/step - loss: 5342.1191 - mse: 5342.1191 - lr: 0.0010
Conv1d model fit done
Predict test set done
Compute metrics done
Mean results of 2 times iteration:
Mean mse of 2 times iteration: 5195.464649509742
Mean rmse of 2 times iteration: 72.07951817793906
Mean mae of 2 times iteration: 71.70519863390373
Mean r2 of 2 times iteration: -125.03111540963687


In [14]:
deep_learning_iteration(model_name='lstm', N_iter=2)

*******************
Iter 0 Start:
LSTM model define done
Epoch 1/2
4/4 [==============================] - 3s 233ms/step - loss: 1.0007 - mse: 1.0007
Epoch 2/2
4/4 [==============================] - 1s 220ms/step - loss: 1.0000 - mse: 1.0000
LSTM model fit done
Predict test set done
Compute metrics done
Iter 1 Start:
LSTM model define done
Epoch 1/2
4/4 [==============================] - 3s 223ms/step - loss: 1.0005 - mse: 1.0005
Epoch 2/2
4/4 [==============================] - 1s 207ms/step - loss: 1.0002 - mse: 1.0002
LSTM model fit done
Predict test set done
Compute metrics done
Mean results of 2 times iteration:
Mean mse of 2 times iteration: 42.01405375946345
Mean rmse of 2 times iteration: 6.481824474000263
Mean mae of 2 times iteration: 5.023491885727391
Mean r2 of 2 times iteration: -0.019173147234344712


In [15]:
deep_learning_iteration(model_name='rnn', N_iter=2)

*******************
Iter 0 Start:
RNN model define done
Epoch 1/2
4/4 [==============================] - 1s 69ms/step - loss: 0.9452 - mse: 0.9452
Epoch 2/2
4/4 [==============================] - 0s 71ms/step - loss: 0.8512 - mse: 0.8512
RNN model fit done
Predict test set done
Compute metrics done
Iter 1 Start:
RNN model define done
Epoch 1/2
4/4 [==============================] - 1s 71ms/step - loss: 1.2494 - mse: 1.2494
Epoch 2/2
4/4 [==============================] - 0s 71ms/step - loss: 0.9634 - mse: 0.9634
RNN model fit done
Predict test set done
Compute metrics done
Mean results of 2 times iteration:
Mean mse of 2 times iteration: 37.91499779324633
Mean rmse of 2 times iteration: 6.154520610989001
Mean mae of 2 times iteration: 4.828670220170455
Mean r2 of 2 times iteration: 0.08026138468911292


In [16]:
deep_learning_iteration(model_name='gru', N_iter=2)

*******************
Iter 0 Start:
GRU model define done
Epoch 1/2
8/8 [==============================] - 2s 121ms/step - loss: 5381.0063 - mse: 5381.0063
Epoch 2/2
8/8 [==============================] - 1s 121ms/step - loss: 5367.4473 - mse: 5367.4473
GRU model fit done
Predict test set done
Compute metrics done
Iter 1 Start:
GRU model define done
Epoch 1/2
8/8 [==============================] - 2s 122ms/step - loss: 5380.0356 - mse: 5380.0356
Epoch 2/2
8/8 [==============================] - 1s 121ms/step - loss: 5363.1787 - mse: 5363.1787
GRU model fit done
Predict test set done
Compute metrics done
Mean results of 2 times iteration:
Mean mse of 2 times iteration: 5226.334952087154
Mean rmse of 2 times iteration: 72.29339123048668
Mean mae of 2 times iteration: 71.98445910076113
Mean r2 of 2 times iteration: -125.77996444034507


In [17]:
deep_learning_iteration(model_name='bilstm', N_iter=2)

*******************
Iter 0 Start:
BiLSTM model define done
Epoch 1/2
4/4 [==============================] - 5s 368ms/step - loss: 0.9919 - mse: 0.9919
Epoch 2/2
4/4 [==============================] - 1s 361ms/step - loss: 0.8549 - mse: 0.8549
BiLSTM model fit done
Predict test set done
Compute metrics done
Iter 1 Start:
BiLSTM model define done
Epoch 1/2
4/4 [==============================] - 5s 366ms/step - loss: 0.9882 - mse: 0.9882
Epoch 2/2
4/4 [==============================] - 1s 364ms/step - loss: 0.7944 - mse: 0.7944
BiLSTM model fit done
Predict test set done
Compute metrics done
Mean results of 2 times iteration:
Mean mse of 2 times iteration: 32.42608789599427
Mean rmse of 2 times iteration: 5.694186096445026
Mean mae of 2 times iteration: 4.427981614805279
Mean r2 of 2 times iteration: 0.21341086859503244
